# Load/Merge raw data and prepare Clustering

In [1]:
import pandas as pd
# read PostalCode/Borough/Neighborhood from already created data
df_pbn = pd.read_csv("PostalCode_Borough_Neighborhood.csv",index_col=0)
# read geo data from provided csv
df_geo = pd.read_csv("Geospatial_Coordinates.csv")
# renaming columns for merge
df_geo = df_geo.rename(columns = {"Postal Code": "PostalCode"})

In [2]:
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
df_pbn.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [4]:
# join on PostalCode
merged_df = pd.merge(df_pbn,df_geo,on="PostalCode")

In [5]:
merged_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# Perform Clustering
The following is left to be done:
- Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.
- Just make sure to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together.

## 1. Create map to get an overview

In [6]:
# import libraries
from sklearn.cluster import KMeans
import folium
import requests # library to handle requests

# use location data from wikipedia to create map of toronto
map_toronto = folium.Map(location=[43.741667, -79.373333], zoom_start=10)

# add markers of neighborhoods/boroughs
for index, row in merged_df.iterrows():
    label = '{}, {}'.format(row['Neighborhood'],row['Borough'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row['Latitude'],row['Longitude']],
        radius=5,
        popup=label,
        colors='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)



In [7]:
# display map
map_toronto


## 2. Get data for nearby venues

In [8]:
# define folium credentials/version
CLIENT_ID = "O4PFHUAQBCNEOAYFSMBDVOUOO2EA3BIZNLH2CTZ4LTE5YJYI"
CLIENT_SECRET = "VH2BSYPSXUM3N0B5ZGNCHTGNU4I3L12A1PRRHJIVWTIUIJAR"
VERSION = '20180605'


In [9]:
# define function to get nearby venues for a borough/langitude+longitude
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
# reload object via pickle if available
import pickle 
toronto_venues = ""
try:
    fh_venues = open('toronto_venues_pickle.obj', 'rb')
    toronto_venues = pickle.load(fh_venues)
except FileNotFoundError:
    toronto_venues = getNearbyVenues(names=merged_df['Neighborhood'],
                                   latitudes=merged_df['Latitude'],
                                   longitudes=merged_df['Longitude']
    )
    # save object to file
    fh_venues = open('toronto_venues_pickle.obj', 'wb')
    pickle.dump(toronto_venues, fh_venues)
fh_venues.close()    

In [11]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


## 3. Analyze data

In [12]:
# Analyze each neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.0,0.033333,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.100000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood,Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.052632,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000


In [14]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0           Steakhouse  0.10
1     Asian Restaurant  0.07
2                 Café  0.07
3                Hotel  0.07
4           Food Court  0.03
5  Monument / Landmark  0.03
6                  Bar  0.03
7               Lounge  0.03
8            Speakeasy  0.03
9         Cocktail Bar  0.03


----Agincourt----
                      venue  freq
0                    Lounge   0.2
1            Sandwich Place   0.2
2            Breakfast Spot   0.2
3              Skating Rink   0.2
4        Chinese Restaurant   0.2
5            Medical Center   0.0
6  Mediterranean Restaurant   0.0
7               Men's Store   0.0
8             Metro Station   0.0
9        Mexican Restaurant   0.0


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                      venue  freq
0                      Park  0.67
1                Playground  0.33
2                     Motel  0.00
3                    Market  0.00
4            Massage S

In [15]:
import numpy as np
# function to get most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

# create dataframe to display top 10 venues for each neighborhood

num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Café,Asian Restaurant,Hotel,Opera House,Monument / Landmark,Concert Hall,Pizza Place,Plaza,Sushi Restaurant
1,Agincourt,Lounge,Chinese Restaurant,Skating Rink,Breakfast Spot,Sandwich Place,Women's Store,Deli / Bodega,Dumpling Restaurant,Drugstore,Dog Run
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Sandwich Place,Fast Food Restaurant,Beer Store,Coffee Shop,Video Store,Creperie
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Gym,Sandwich Place,Pub,Skating Rink,Pharmacy,Dog Run,Discount Store,Curling Ice


## 4. Cluster data

In [16]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 8, 2, 2, 2, 0, 2, 0, 0])

Create dataframe that includes cluster+top 10 venues

In [17]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = merged_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Fast Food Restaurant,Food & Drink Shop,Bus Stop,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Performing Arts Venue,Chocolate Shop
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Athletics & Sports,Event Space,Miscellaneous Shop,Coffee Shop,Shoe Store,Boutique,Women's Store,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Diner,Japanese Restaurant,Gym,Burger Joint,Yoga Studio,Creperie,Seafood Restaurant,Smoothie Shop,Nightclub
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,4.0,Fast Food Restaurant,Women's Store,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
7,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Gym / Fitness Center,Japanese Restaurant,Caribbean Restaurant,Café,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,2.0,Fast Food Restaurant,Pizza Place,Pet Store,Rock Climbing Spot,Intersection,Bank,Athletics & Sports,Pharmacy,Gastropub,Gym / Fitness Center
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0.0,Café,Clothing Store,Gastropub,Burrito Place,Hotel,Beer Bar,Shopping Mall,Japanese Restaurant,Sandwich Place,Diner


In [18]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import math

# visualize resulting clusters
map_toronto_clusters = folium.Map(location=[43.741667, -79.373333], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if(math.isnan(cluster)):
        continue
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_toronto_clusters)
       
map_toronto_clusters